# CARTO & BigQuery

CartoFrames plays nicely with Google BigQuery! Just run a query, convert to a GeoDataFrame and let the data speak for itself.

Here, we're looking at differences in median income in Kings County (Brooklyn) between 2010 and 2017. Historical ACS data (2007 - 2018) is provided by CARTO and made publicly available as part of [BigQuery's Public Datasets](https://cloud.google.com/bigquery/public-data/).

For visualization we're joining with [CARTO's custom shoreline-clipped version](https://carto.com/blog/tiger-shoreline-clip/) of the U.S. Census' TIGER boundaries. These are also publicly accessible as part of CARTO's own public dataset initiative in the `carto-do-public-data` BQ project.

In [1]:
import os

import geopandas as gpd

from cartoframes.viz.helpers import color_continuous_layer
from google.cloud import bigquery
from shapely import wkt

In [2]:
os.environ['GOOGLE_CLOUD_PROJECT'] = 'YOUR_GOOGLE_CLOUD_PROJECT'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'YOUR_SERVICE_ACCOUNT.json'

bq_client = bigquery.Client()

In [3]:
q = '''
WITH acs_2017 AS (
  SELECT geoid, median_income AS median_income_2017
  FROM `bigquery-public-data-staging.cartodb_test_env.census_acs_blockgroup_2017_5yr`  
  WHERE geoid LIKE '36047%'
),

acs_2010 AS (
  SELECT geoid, median_income AS median_income_2010
  FROM `bigquery-public-data-staging.cartodb_test_env.census_acs_blockgroup_2010_5yr` 
  WHERE geoid LIKE '36047%'
),

acs_diff AS (
  SELECT
    a17.geoid, a17.median_income_2017, a10.median_income_2010, geo.geom,
    a17.median_income_2017 - a10.median_income_2010 AS median_income_diff
  FROM acs_2017 a17
  JOIN acs_2010 a10
    ON a17.geoid = a10.geoid
  JOIN `carto-do-public-data.open_data.geography_usa_blockgroupclipped_2015` geo
    ON a17.geoid = geo.geoid
)

SELECT * FROM acs_diff WHERE median_income_diff IS NOT NULL
'''

df = bq_client.query(q).to_dataframe()
gdf = gpd.GeoDataFrame(df, geometry=df.geom.apply(wkt.loads))

color_continuous_layer(gdf, value='median_income_diff', widget=True)